# 系统上线情况分析

In [26]:
import pandas as pd
import numpy as np

## 数据导入

文件说明:

- ./data/administrative_divisions.xlsx: 行政区划代码表, 目前只记录了与园区有关的行政区域
  - code: 行政区划代码
  - level: 级别. 0-中国 1-省份 2-地市 3-区县
  - pid: 父节点 id
- ./data/organization.xlsx: 组织机构表, 不含项目部级, 项目部信息在 iindustry_parks.xlsx 文件中
  - level: 级别. 0-事业部 1-分公司
  - territory_id: 归属地 id. 关联 administrative_divisions.xlsx 文件中的 id
  - pid: 父节点 id
- ./data/industry_parks.xlsx: 园区信息/项目部信息
  - org_id: 所属组织 id. 关联 organization.xlsx 文件中的 id
  - status: 园区状态. [筹开期, 在建期, 待建期, 成熟期, 培育期]
  - management_type: 管理类型. [垂直管理, 专业赋能]
  - territory_id: 归属地 id. 关联 administrative_divisions.xlsx 文件中的 id. 单独设置该字段而不采用 "所属组织(org_id)" 的归属地, 是因为项目的归属地与项目所属组织的归属地会不同.
  - product_deparment: 产品系. [网谷系, 意库系, 智慧城及其他, 创库系]
- ./data/app_usage.xlsx: 系统使用情况
  - ip_id: 园区 id. 关联 industry_parks.xlsx 文件中 id.
  - ip_name: 园区名称. 冗余字段, 该字段是为了直接打开 excel 文件时容易编辑, 后续生成分析报表时, 应从 industry_parks.xlsx 文件中关联读取.
  - app_name: 系统名称. [InPark, 小招通, 租赁平台, 产业通一期, 园区智能化(IOC), BI数据治理]
  - usage_status: 使用情况. [不需要, 已上线, 未上线, 上线中, Pending, Waiting]. Pending-系统还未研发完成 Waiting-园区还未开业


In [71]:
# load 行政代码表
df_divisions = pd.read_excel('./data/administrative_divisions.xlsx')
# load 组织机构表
df_org = pd.read_excel('./data/organization.xlsx')
# load 园区信息表
df_park_origin = pd.read_excel('./data/industry_parks.xlsx')
# load 系统使用情况表
df_usage_origin = pd.read_excel('./data/app_usage.xlsx')

## 数据清洗

处理园区信息表中的关联字段

In [78]:
# 关联组织
# df_org.iloc[:, :2]: 只从 组织机构表 中取头两列, 即 id, name
df_park = df_park_origin.merge(
    df_org.iloc[:, :2], left_on='org_id', right_on='id')

# 从 df 中 去除不需要的字段
# np.setdiff1d 函数的应用起到数组取补集的作用
no_need_cols = ['org_id', 'id_y']
df_park = df_park.loc[:, np.setdiff1d(df_park.columns, no_need_cols)]

# 关联归属地
# df_divisions[['id', 'name']]: 只从 行政区划表 中取 'id' 和 'name' 两列
#   作用和上面的 df_org.iloc[:, :2] 作用一样, 只是换个写法
df_park = df_park.merge(
    df_divisions[['id', 'name']], left_on='territory_id', right_on='id')
# 从 df 中 去除不需要的字段
no_need_cols = ['id', 'territory_id']
df_park = df_park.loc[:, np.setdiff1d(df_park.columns, no_need_cols)]

# rename cols
df_park = df_park.rename(columns={
    'id_x': 'id',
    'name': 'territory_name',
    'name_x': 'ip_name',
    'name_y': 'org_name',
})

# 调整字段顺序
df_park = df_park[['id', 'ip_name', 'management_type',
                   'product_department', 'status', 'org_name', 'territory_name']]
df_park

,id,ip_name,management_type,product_department,status,org_name,territory_name
0,1,城北创库,专业垂直,创库系,筹开期,杭州公司,杭州市
1,5,信雅达创库,专业垂直,创库系,成熟期,杭州公司,杭州市
2,6,运河网谷,专业垂直,网谷系,筹开期,杭州公司,杭州市
3,2,南通豪华游轮配套产业园,专业垂直,智慧城及其他,在建期,杭州公司,南通市
4,3,宁波潜龙创库,专业垂直,创库系,待建期,杭州公司,宁波市
5,4,上海森兰美奂创库,专业垂直,创库系,成熟期,杭州公司,上海市
6,7,创马项目,专业垂直,网谷系,在建期,南京公司,南京市
7,8,高铁网谷,专业垂直,网谷系,成熟期,南京公司,南京市
8,11,紫金智谷,专业垂直,网谷系,成熟期,南京公司,南京市
9,9,合肥庐州意库,专业垂直,意库系,培育期,南京公司,合肥市


系统使用情况表和园区信息表关联

In [79]:
# 关联 园区信息表
# df_usage_origin[np.setdiff1d(df_usage_origin.columns, ['ip_name'])]
#   去除 系统使用情况表 的 'ip_name' 字段后再进行管理
#   因为 '园区名称(ip_name)' 应该从 园区信息表 中获取
df_usage = df_usage_origin[np.setdiff1d(df_usage_origin.columns, ['ip_name'])].merge(
    df_park, left_on='ip_id', right_on='id')
# 从 df 中 去除不需要的字段
no_need_cols = ['ip_id', 'id_y']
df_usage = df_usage.loc[:, np.setdiff1d(df_usage.columns, no_need_cols)]
# rename cols
df_usage = df_usage.rename(columns={
    'name': 'ip_name',
    'id_x': 'id'
})
# rearrange cols
df_usage = df_usage[['id', 'ip_name', 'management_type',
                     'product_department', 'status', 'org_name', 'territory_name', 'app_name', 'usage_status', 'memo']]

df_usage

,id,ip_name,management_type,product_department,status,org_name,territory_name,app_name,usage_status,memo
0,0,城北创库,专业垂直,创库系,筹开期,杭州公司,杭州市,InPark,不需要,NaN
1,1,城北创库,专业垂直,创库系,筹开期,杭州公司,杭州市,小招通,已上线,NaN
2,2,城北创库,专业垂直,创库系,筹开期,杭州公司,杭州市,租赁平台,不需要,NaN
3,3,城北创库,专业垂直,创库系,筹开期,杭州公司,杭州市,产业通一期,未上线,NaN
4,4,城北创库,专业垂直,创库系,筹开期,杭州公司,杭州市,园区智能化（IOC）,不需要,NaN
...,...,...,...,...,...,...,...,...,...,...
157,157,番禺科技园,专业垂直,智慧城及其他,成熟期,番禺科技园,广州市,小招通,不需要,NaN
158,158,番禺科技园,专业垂直,智慧城及其他,成熟期,番禺科技园,广州市,租赁平台,已上线,NaN
159,159,番禺科技园,专业垂直,智慧城及其他,成熟期,番禺科技园,广州市,产业通一期,已上线,NaN
160,160,番禺科技园,专业垂直,智慧城及其他,成熟期,番禺科技园,广州市,园区智能化（IOC）,不需要,NaN
